# Trying to upload DICOMs and ROIs (DICOM-RTSTRUCTs) to XNAT using XNATpy

### https://xnat.readthedocs.io/en/latest/index.html

### Changes from v1:  
#### - Upload all DICOMs whose UIDs correspond to the UIDs of the ROI data.  
#### - May need to modify the Project/Subject/Experiment(Session) labels in the DICOM data and RTSTRUCT file to match the labels assigned when creating new Project/Subject/Experiment...

In [8]:
import xnat
#xnat.__version__ # 0.3.21
print(f'XNATpy version = {xnat.__version__}') # 0.3.21
import SimpleITK as sitk
import pydicom
import dicom2nifti
import numpy as np
#import math
import matplotlib.pyplot as plt
#from matplotlib import pyplot
import matplotlib.image as mpimg
import os, sys, time
import natsort
import dicom_functions
import explore_dicom_data
from myshow import myshow
import importlib

XNATpy version = 0.3.21


#### Connect to XNAT server:

In [9]:
xnatAddress = 'http://10.1.1.17'
#xnatAddress = 'http://10.1.1.17/app/template/Index.vm'
#xnatAddress = 'http://10.1.1.17/app/template/Login.vm#!'
#xnatAddress = 'http://10.1.1.17:8080/xnat'
#xnatAddress = 'http://10.1.1.17:80/xnat'
#xnatAddress = 'http://localhost:8080/xnat'
#xnatAddress = 'http://10.1.1.17:8080/admin'
#xnatAddress = 'http://localhost:8080/admin'

#session = xnat.connect(xnatAddress, user='xnat', password='xnat')
#session = xnat.connect(xnatAddress, user='owner', password='owner')
session = xnat.connect(xnatAddress, user='admin', password='admin', debug=True)

[2020-02-17 08:50:18,857] DEBUG: __init__:197 >> Response url: http://10.1.1.17/data/projects
[2020-02-17 08:50:18,858] DEBUG: __init__:59 >> Getting http://10.1.1.17/ to test auth
[2020-02-17 08:50:19,072] DEBUG: __init__:61 >> Status: 200
[2020-02-17 08:50:19,073] INFO: __init__:95 >> Logged in successfully as admin
[2020-02-17 08:50:19,074] DEBUG: session:334 >> GET URI http://10.1.1.17/data/JSESSION
[2020-02-17 08:50:19,086] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 08:50:19,087] DEBUG: session:334 >> GET URI http://10.1.1.17/data/version
[2020-02-17 08:50:19,093] DEBUG: session:307 >> Received response with status code: 404
[2020-02-17 08:50:19,094] DEBUG: session:334 >> GET URI http://10.1.1.17/xapi/siteConfig/buildInfo?format=json
[2020-02-17 08:50:19,099] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 08:50:19,100] INFO: __init__:239 >> Found an 1.7 version (1.7.5.6)
[2020-02-17 08:50:19,101] DEBUG: session:334 >> GET U

[2020-02-17 08:50:19,225] DEBUG: convert_xsd:1237 >> Adding EEGSession -> xnat:eegSessionData to class name map
[2020-02-17 08:50:19,226] DEBUG: convert_xsd:1237 >> Adding OtherDicomSession -> xnat:otherDicomSessionData to class name map
[2020-02-17 08:50:19,228] DEBUG: convert_xsd:1237 >> Adding SubjectVariables -> xnat:subjectVariablesData to class name map
[2020-02-17 08:50:19,228] DEBUG: convert_xsd:1237 >> Adding PETSession -> xnat:petSessionData to class name map
[2020-02-17 08:50:19,229] DEBUG: convert_xsd:1237 >> Adding PETMRSession -> xnat:petmrSessionData to class name map
[2020-02-17 08:50:19,230] DEBUG: convert_xsd:1237 >> Adding MRScan -> xnat:mrScanData to class name map
[2020-02-17 08:50:19,230] DEBUG: convert_xsd:1237 >> Adding CTScan -> xnat:ctScanData to class name map
[2020-02-17 08:50:19,231] DEBUG: convert_xsd:1237 >> Adding PETScan -> xnat:petScanData to class name map
[2020-02-17 08:50:19,232] DEBUG: convert_xsd:1237 >> Adding EPSScan -> xnat:epsScanData to class

[2020-02-17 08:50:19,285] DEBUG: convert_xsd:932 >> Found 3 unknown tags: {'{http://nrg.wustl.edu/xdat}relation', '{http://nrg.wustl.edu/security}element', '{http://nrg.wustl.edu/security}field'}
[2020-02-17 08:50:19,286] INFO: convert_xsd:949 >> === Retrieving schema from /xapi/schemas/assessments ===
[2020-02-17 08:50:19,287] DEBUG: session:334 >> GET URI http://10.1.1.17/xapi/schemas/assessments
[2020-02-17 08:50:19,310] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 08:50:19,313] DEBUG: convert_xsd:919 >> Registering namespace: xs  ->  http://www.w3.org/2001/XMLSchema
[2020-02-17 08:50:19,314] DEBUG: convert_xsd:919 >> Registering namespace: xnat  ->  http://nrg.wustl.edu/xnat
[2020-02-17 08:50:19,314] DEBUG: convert_xsd:919 >> Registering namespace: xnat_a  ->  http://nrg.wustl.edu/xnat_assessments
[2020-02-17 08:50:19,314] DEBUG: convert_xsd:1241 >> Skipping non-class top-level tag {http://www.w3.org/2001/XMLSchema}import
[2020-02-17 08:50:19,315] DEBUG

[2020-02-17 08:50:19,432] DEBUG: convert_xsd:919 >> Registering namespace: arc  ->  http://nrg.wustl.edu/arc
[2020-02-17 08:50:19,433] DEBUG: convert_xsd:919 >> Registering namespace: xs  ->  http://www.w3.org/2001/XMLSchema
[2020-02-17 08:50:19,433] DEBUG: convert_xsd:919 >> Registering namespace: xdat  ->  http://nrg.wustl.edu/xdat
[2020-02-17 08:50:19,434] DEBUG: convert_xsd:1237 >> Adding ArchiveSpecification -> None to class name map
[2020-02-17 08:50:19,435] DEBUG: convert_xsd:932 >> Found 3 unknown tags: {'{http://nrg.wustl.edu/xdat}relation', '{http://nrg.wustl.edu/security}element', '{http://nrg.wustl.edu/security}field'}
[2020-02-17 08:50:19,435] INFO: convert_xsd:949 >> === Retrieving schema from /xapi/schemas/validation/protocolValidation ===
[2020-02-17 08:50:19,436] DEBUG: session:334 >> GET URI http://10.1.1.17/xapi/schemas/validation/protocolValidation
[2020-02-17 08:50:19,445] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 08:50:19,446] DEBUG

[2020-02-17 08:50:19,516] INFO: convert_xsd:1356 >> Removing class xnatpy:roiCollectionDataReferences from parser, it only has one element!
[2020-02-17 08:50:19,517] INFO: convert_xsd:1356 >> Removing class xnatpy:role_typeAllowed_actions from parser, it only has one element!
[2020-02-17 08:50:19,517] INFO: convert_xsd:1356 >> Removing class xnatpy:element_securityPrimary_security_fields from parser, it only has one element!
[2020-02-17 08:50:19,518] INFO: convert_xsd:1356 >> Removing class xnatpy:element_securityElement_actions from parser, it only has one element!
[2020-02-17 08:50:19,518] INFO: convert_xsd:1356 >> Removing class xnatpy:element_securityListing_actions from parser, it only has one element!
[2020-02-17 08:50:19,519] DEBUG: convert_xsd:1362 >> Ignoring non-listing...
[2020-02-17 08:50:19,519] DEBUG: convert_xsd:1363 >> Element class: {'parser': <xnat.convert_xsd.SchemaParser object at 0x0000013ADA0E7348>, 'name': 'xnatpy:userPrimary_password', 'logger': <Logger xnat-431

...

[2020-02-17 08:50:19,743] INFO: convert_xsd:1010 >> Processing xnat:ioScanData (base class xnat:imageScanData)
[2020-02-17 08:50:19,744] INFO: convert_xsd:1010 >> Processing xnat:usSessionData (base class xnat:imageSessionData)
[2020-02-17 08:50:19,744] INFO: convert_xsd:1010 >> Processing xnat:usScanData (base class xnat:imageScanData)
[2020-02-17 08:50:19,745] INFO: convert_xsd:1010 >> Processing xnat:ecgSessionData (base class xnat:imageSessionData)
[2020-02-17 08:50:19,745] INFO: convert_xsd:1010 >> Processing xnat:ecgScanData (base class xnat:imageScanData)
[2020-02-17 08:50:19,746] INFO: convert_xsd:1010 >> Processing xnat:hdSessionData (base class xnat:imageSessionData)
[2020-02-17 08:50:19,746] INFO: convert_xsd:1010 >> Processing xnat:hdScanData (base class xnat:imageScanData)
[2020-02-17 08:50:19,746] INFO: convert_xsd:1010 >> Processing xnat:epsSessionData (base class xnat:imageSessionData)
[2020-02-17 08:50:19,747] INFO: convert_xsd:1010 >> Processing xnat:epsScanData (base

[2020-02-17 08:50:19,775] INFO: convert_xsd:1010 >> Processing xnatpy:eegScanDataParametersDatarecord (base class None)
[2020-02-17 08:50:19,776] INFO: convert_xsd:1010 >> Processing xnatpy:eegScanDataChannels (base class None)
[2020-02-17 08:50:19,777] INFO: convert_xsd:1010 >> Processing xnatpy:eegScanDataSoftwarefiltersimpedances (base class None)
[2020-02-17 08:50:19,778] INFO: convert_xsd:1010 >> Processing xnatpy:eegScanDataSoftwarefiltersimpedancesImpedance (base class None)
[2020-02-17 08:50:19,779] INFO: convert_xsd:1010 >> Processing xnat_a:sideEffectsPittsburghData (base class xnat:subjectAssessorData)
[2020-02-17 08:50:19,780] INFO: convert_xsd:1010 >> Processing xnat_a:ybocsData (base class xnat:subjectAssessorData)
[2020-02-17 08:50:19,782] INFO: convert_xsd:1010 >> Processing xnat_a:ygtssData (base class xnat:subjectAssessorData)
[2020-02-17 08:50:19,783] INFO: convert_xsd:1010 >> Processing xnatpy:ygtssDataMotor (base class None)
[2020-02-17 08:50:19,784] INFO: convert_

[2020-02-17 08:50:19,835] INFO: convert_xsd:1010 >> Processing arc:pipelineParameterData (base class None)
[2020-02-17 08:50:19,836] DEBUG: convert_xsd:997 >> Wait with processing xnatpy:pipelineParameterDataCsvvalues because base xnatpy:csvValuesString is not yet processed
[2020-02-17 08:50:19,837] INFO: convert_xsd:1010 >> Processing xnatpy:csvValuesString (base class None)
[2020-02-17 08:50:19,837] INFO: convert_xsd:1010 >> Processing arc:pathInfo (base class None)
[2020-02-17 08:50:19,838] DEBUG: convert_xsd:997 >> Wait with processing arc:property because base xnatpy:propertyString is not yet processed
[2020-02-17 08:50:19,838] INFO: convert_xsd:1010 >> Processing xnatpy:propertyString (base class None)
[2020-02-17 08:50:19,838] DEBUG: convert_xsd:997 >> Wait with processing arc:fieldSpecification because base xnatpy:fieldSpecificationString is not yet processed
[2020-02-17 08:50:19,838] INFO: convert_xsd:1010 >> Processing xnatpy:fieldSpecificationString (base class None)
[2020-0

[2020-02-17 08:50:19,900] INFO: convert_xsd:1010 >> Processing xnat:petAssessorData (base class xnat:imageAssessorData)
[2020-02-17 08:50:19,903] INFO: convert_xsd:1010 >> Processing xnatpy:petSessionDataTracerDose (base class xnatpy:doseFloat)
[2020-02-17 08:50:19,904] INFO: convert_xsd:1010 >> Processing xnatpy:petSessionDataTracerTotalmass (base class xnatpy:totalMassFloat)
[2020-02-17 08:50:19,905] INFO: convert_xsd:1010 >> Processing xnatpy:petSessionDataTracerIntermediate (base class xnatpy:intermediateFloat)
[2020-02-17 08:50:19,906] INFO: convert_xsd:1010 >> Processing xnatpy:petSessionDataTracerIsotope (base class xnatpy:isotopeString)
[2020-02-17 08:50:19,907] INFO: convert_xsd:1010 >> Processing xnat:petScanData (base class xnat:imageScanData)
[2020-02-17 08:50:19,908] INFO: convert_xsd:1010 >> Processing xnatpy:petScanDataParameters (base class None)
[2020-02-17 08:50:19,911] INFO: convert_xsd:1010 >> Processing xnatpy:petScanDataParametersFrames (base class None)
[2020-02-

[2020-02-17 08:50:19,935] INFO: convert_xsd:1021 >> Missed: []
[2020-02-17 08:50:19,936] INFO: convert_xsd:1022 >> Missed base class: []
[2020-02-17 08:50:19,936] DEBUG: convert_xsd:1023 >> Spent 2 iterations
[2020-02-17 08:50:19,952] DEBUG: __init__:249 >> Code file written to: C:\Users\ctorti\AppData\Local\Temp\tmpz38cnag9_generated_xnat.py
[2020-02-17 08:50:20,109] DEBUG: __init__:256 >> Loaded generated module


In [4]:
session

#### Define the REST path variables:

In [10]:
# Define new project and subject labels, and session number:
projectLabel = 'BrainTumorProg_copy'
subjectLabel = 'PGM-002_Feb17'
experimentLabel = subjectLabel + '_MR_1'
#experimentNo = 0 # (= 1 in XNAT web app; e.g. ‘PGM-002_MR_1’)
#scanNo = 0 # 0 -> T1post
print('Project label =', projectLabel)
print('Subject label =', subjectLabel)
print('Experiment label =', experimentLabel)

Project label = BrainTumorProg_copy
Subject label = PGM-002_Feb17
Experiment label = PGM-002_Feb17_MR_1


#### Location of previously downloaded DICOM and DICOM-RTSTRUCT files:

In [32]:
# The DICOM files were exported here:
dicomDLdir = r'C:\Code\WP1.3_multiple_modalities\trying stuff\XNAT downloads\BrainTumorProg\PGM-002\0\0\PGM-002_MR_1\scans\11-T1post\resources\DICOM\files'

# And the ROI DICOM-RTStruct file was exported here:
roiDLdir = r'C:\Code\WP1.3_multiple_modalities\trying stuff\XNAT downloads\BrainTumorProg\PGM-002\0\0\AIM_20200123_145716\out\resources\RTSTRUCT\files'

#### Create a new project in XNAT:

In [14]:
project = session.classes.ProjectData(parent=session, name=projectLabel)

[2020-02-17 09:01:25,520] DEBUG: core:171 >> PUT URI: /data/archive/projects/BrainTumorProg_copy
[2020-02-17 09:01:25,521] DEBUG: core:180 >> query: {'xsiType': 'xnat:projectData', 'req_format': 'qs', 'name': 'BrainTumorProg_copy'}
[2020-02-17 09:01:25,522] DEBUG: session:428 >> PUT URI http://10.1.1.17/data/archive/projects/BrainTumorProg_copy?xsiType=xnat%3AprojectData&req_format=qs&name=BrainTumorProg_copy
[2020-02-17 09:01:25,523] DEBUG: session:430 >> PUT DATA None
[2020-02-17 09:01:25,523] DEBUG: session:431 >> PUT FILES None
[2020-02-17 09:01:25,631] DEBUG: session:307 >> Received response with status code: 200


#### Create a new subject for the new project:

In [15]:
subject = session.classes.SubjectData(parent=project, label=subjectLabel)

[2020-02-17 09:03:45,692] DEBUG: core:171 >> PUT URI: /data/archive/projects/BrainTumorProg_copy/subjects/PGM-002_Feb17
[2020-02-17 09:03:45,693] DEBUG: core:180 >> query: {'xsiType': 'xnat:subjectData', 'req_format': 'qs', 'label': 'PGM-002_Feb17'}
[2020-02-17 09:03:45,694] DEBUG: session:428 >> PUT URI http://10.1.1.17/data/archive/projects/BrainTumorProg_copy/subjects/PGM-002_Feb17?xsiType=xnat%3AsubjectData&req_format=qs&label=PGM-002_Feb17
[2020-02-17 09:03:45,694] DEBUG: session:430 >> PUT DATA None
[2020-02-17 09:03:45,695] DEBUG: session:431 >> PUT FILES None
[2020-02-17 09:03:45,912] DEBUG: session:307 >> Received response with status code: 201


#### Create a new experiment for the new subject:

##### Note:  Spaces are not permitted for labels!

In [17]:
experiment = session.classes.MrSessionData(parent=subject, label=experimentLabel)

[2020-02-17 09:05:30,706] DEBUG: session:334 >> GET URI http://10.1.1.17/data/archive/projects/BrainTumorProg_copy/subjects/PGM-002_Feb17?format=json
[2020-02-17 09:05:30,759] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 09:05:30,760] DEBUG: core:171 >> PUT URI: /data/archive/projects/BrainTumorProg_copy/subjects/XNAT_S00008/experiments/PGM-002_Feb17_MR_1
[2020-02-17 09:05:30,761] DEBUG: core:180 >> query: {'xsiType': 'xnat:mrSessionData', 'req_format': 'qs', 'label': 'PGM-002_Feb17_MR_1'}
[2020-02-17 09:05:30,761] DEBUG: session:428 >> PUT URI http://10.1.1.17/data/archive/projects/BrainTumorProg_copy/subjects/XNAT_S00008/experiments/PGM-002_Feb17_MR_1?xsiType=xnat%3AmrSessionData&req_format=qs&label=PGM-002_Feb17_MR_1
[2020-02-17 09:05:30,762] DEBUG: session:430 >> PUT DATA None
[2020-02-17 09:05:30,762] DEBUG: session:431 >> PUT FILES None
[2020-02-17 09:05:30,936] DEBUG: session:307 >> Received response with status code: 201


#### Load the RTSTRUCT file to the get the Referenced SOP Instance UIDs

In [28]:
# Get the filename of the DICOM-RTSTRUCT file:
roiFname = os.listdir(roiDLdir)
roiFname = roiFname[0]

roiFpath = os.path.join(roiDLdir, roiFname)

# Load the DICOM-RTStruct file:
roi = pydicom.dcmread(roiFpath)

roi

(0008, 0016) SOP Class UID                       UI: RT Structure Set Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.534147.578.4208788039.20200231457312
(0008, 0020) Study Date                          DA: '19960813'
(0008, 0030) Study Time                          TM: '162908'
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'RTSTRUCT'
(0008, 0070) Manufacturer                        LO: 'GE MEDICAL SYSTEMS'
(0008, 0090) Referring Physician's Name          PN: ''
(0008, 1070) Operators' Name                     PN: ''
(0008, 1090) Manufacturer's Model Name           LO: 'Optima MR450w'
(0010, 0010) Patient's Name                      PN: 'PGM-002'
(0010, 0020) Patient ID                          LO: 'PGM-002_MR_1'
(0010, 0030) Patient's Birth Date                DA: '00021130'
(0010, 0040) Patient's Sex                       CS: 'M'
(0018, 1020) Software Version(s)                 LO: ['25', 'LX', '

#### Get a list of the Referenced SOP Instance UIDs in roi

In [30]:
roiUids = []

# Loop for each contour sequence:
for sequence in roi.ROIContourSequence[0].ContourSequence:
    roiUids.append(sequence.ContourImageSequence[0].ReferencedSOPInstanceUID)
    
roiUids

['1.3.6.1.4.1.14519.5.2.1.4429.7055.320280293613596547840725755465',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.677643793103545804182040061154',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.149742284257271356119234155647',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.207269894846632100106605315699',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.207269894846632100106605315699',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.337321577215686178552541657021',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.337321577215686178552541657021',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.121761661581454976960293845378',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.121761661581454976960293845378',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.121761661581454976960293845378',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.176309996049760286843247671656',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.176309996049760286843247671656',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.176309996049760286843247671656',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.213859255991588180723164022910',
 '1.3.6.1.4.1.14519.5.2.1.4429.705

#### Read in each DICOM and create a list of indeces that correspond to the DICOMs that have SOP Instance UIDs that match the Reference SOP Instance UIDs in the ROI data

##### Note:  The ordering of inds is not the same as the ordering of the frame number since the list of DICOM files are not read in using natsort!

In [34]:
# Get the filenames of the DICOM files:
dicomFnames = os.listdir(dicomDLdir)
#dicomFnames

# Create a list of full file paths:
dicomFpaths = []

# Initialise array of indices:
inds = []

# Loop for each file name in dicomFnames:
for f in range(len(dicomFnames)):
    dicomFpath = os.path.join(dicomDLdir, dicomFnames[f])
    
    dicomFpaths.append(dicomFpath)
    
    # Read in the dicom file:
    dicom = pydicom.dcmread(dicomFpath)
    
    dicomUid = dicom.SOPInstanceUID
    
    # If dicomUid is in roiUids append f in inds:
    if dicomUid in roiUids:
        inds.append(f)
        
inds

[3, 4, 5, 6, 7, 8, 9, 10, 12]

#### Try uploading data to XNAT

##### Note:  It should be possible to upload an entire directory of files but I'm not sure how, and for the benefit of working out what conditions need to be met to successfully upload an RTSTRUCT file, I'll only upload the DICOMs that have corresponding ROI contour data.

In [46]:
# Upload to the prearchive the DICOMs that have contour data:
for f in range(len(inds)):
    print(f'Uploading file {f+1} of {len(inds)}...')
    prearchive_session = session.services.import_(dicomFpaths[f], \
                                                  project=projectLabel, \
                                                  destination='/prearchive')

[2020-02-17 10:17:20,634] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy


Uploading file 1 of 9...


[2020-02-17 10:17:21,254] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 10:17:21,255] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy


Uploading file 2 of 9...


[2020-02-17 10:17:21,815] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 10:17:21,817] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy


...

[2020-02-17 10:17:24,554] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 10:17:24,555] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy


Uploading file 8 of 9...


[2020-02-17 10:17:25,192] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 10:17:25,193] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy


Uploading file 9 of 9...


[2020-02-17 10:17:25,716] DEBUG: session:307 >> Received response with status code: 200


In [37]:
# Get a list of sessions waiting for archiving:

session.prearchive.sessions()

[2020-02-17 10:04:11,304] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects?format=json
[2020-02-17 10:04:11,339] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 10:04:11,340] DEBUG: session:334 >> GET URI http://10.1.1.17/data/version
[2020-02-17 10:04:11,348] DEBUG: session:307 >> Received response with status code: 404
[2020-02-17 10:04:11,348] DEBUG: session:334 >> GET URI http://10.1.1.17/xapi/siteConfig/buildInfo?format=json
[2020-02-17 10:04:11,358] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 10:04:11,360] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy/20200217_100301217/PGM_002_MR_1?format=json
[2020-02-17 10:04:11,382] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 10:04:11,383] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy/20200206_152623716/PGM_002_MR_1?format=json
[2020-02-17 10:04

[<PrearchiveSession BrainTumorProg_copy/20200217_100301217/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200206_152623716/PGM_002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200207_144303310/PGM_002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200212_213423578/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200212_204058544/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200212_215434501/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_100306307/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200212_161330203/PGM_002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200212_162336865/PGM_002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200214_085602810/PGM-002_Feb14_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_100303842/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_100303240/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_100302596/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorPr

In [40]:
session.prearchive.sessions()[0]

[2020-02-17 10:10:46,202] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects?format=json
[2020-02-17 10:10:46,218] DEBUG: session:307 >> Received response with status code: 200


<PrearchiveSession BrainTumorProg_copy/20200217_100301217/PGM-002_MR_1>

In [45]:
session.prearchive.find(status='READY')

[2020-02-17 10:15:10,193] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects?format=json
[2020-02-17 10:15:10,208] DEBUG: session:307 >> Received response with status code: 200


[<PrearchiveSession BrainTumorProg_copy/20200217_100301217/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_100306307/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200214_085602810/PGM-002_Feb14_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_100303842/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_100303240/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_100302596/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_100301952/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_100304478/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_100305076/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_100305671/PGM-002_MR_1>]

## This time upload to the prearchive assigning not only the project, but also the subject and experiment:

In [74]:
# Upload to the prearchive the DICOMs that have contour data:
for f in range(len(inds)):
    print(f'Uploading file {f+1} of {len(inds)} to the prearchive...')
    
    prearchive_session = session.services.import_(dicomFpaths[inds[f]], \
                                                  project=projectLabel, \
                                                  subject=subjectLabel, \
                                                  experiment=experimentLabel, \
                                                  destination='/prearchive')

[2020-02-17 11:28:33,163] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy&subject=PGM-002_Feb17&session=PGM-002_Feb17_MR_1


Uploading file 1 of 9 to the prearchive...


[2020-02-17 11:28:33,696] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 11:28:33,697] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy&subject=PGM-002_Feb17&session=PGM-002_Feb17_MR_1


Uploading file 2 of 9 to the prearchive...


[2020-02-17 11:28:34,211] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 11:28:34,213] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy&subject=PGM-002_Feb17&session=PGM-002_Feb17_MR_1


...

[2020-02-17 11:28:37,127] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 11:28:37,130] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy&subject=PGM-002_Feb17&session=PGM-002_Feb17_MR_1


Uploading file 8 of 9 to the prearchive...


[2020-02-17 11:28:37,823] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 11:28:37,825] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy&subject=PGM-002_Feb17&session=PGM-002_Feb17_MR_1


Uploading file 9 of 9 to the prearchive...


[2020-02-17 11:28:38,512] DEBUG: session:307 >> Received response with status code: 200


In [75]:
session.prearchive.find(status='READY')

[2020-02-17 11:29:21,939] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects?format=json
[2020-02-17 11:29:21,955] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 11:29:21,957] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy/20200217_112830490/PGM_002_MR_1?format=json
[2020-02-17 11:29:21,972] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 11:29:21,973] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy/20200217_112833178/PGM_002_MR_1?format=json
[2020-02-17 11:29:21,980] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 11:29:21,981] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy/20200217_112831518/PGM_002_MR_1?format=json
[2020-02-17 11:29:21,992] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 11:29:21,993] DEBUG: session:334 >> GET 

[<PrearchiveSession BrainTumorProg_copy/20200217_100301217/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_101718616/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_100306307/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200214_085602810/PGM-002_Feb14_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112830490/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112833178/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112831518/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112829964/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112832580/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112831005/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112832043/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112834623/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112833943

In [76]:
session.prearchive.find(project=projectLabel, subject=subjectLabel, session=experimentLabel, status='READY')

[2020-02-17 11:29:29,304] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy?format=json
[2020-02-17 11:29:29,313] DEBUG: session:307 >> Received response with status code: 200


[<PrearchiveSession BrainTumorProg_copy/20200217_112830490/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112833178/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112831518/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112829964/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112832580/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112831005/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112832043/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112834623/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112833943/PGM-002_Feb17_MR_1>]

In [77]:
# Create a session of the files within the prearchive that I want to archive:
prearchive_session_to_archive = session.prearchive.find(project=projectLabel, \
                                                        subject=subjectLabel, \
                                                        session=experimentLabel, \
                                                        status='READY')

[2020-02-17 11:30:05,496] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy?format=json
[2020-02-17 11:30:05,509] DEBUG: session:307 >> Received response with status code: 200


In [78]:
prearchive_session_to_archive

[<PrearchiveSession BrainTumorProg_copy/20200217_112830490/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112833178/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112831518/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112829964/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112832580/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112831005/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112832043/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112834623/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112833943/PGM-002_Feb17_MR_1>]

In [79]:
len(prearchive_session_to_archive)

9

##### the project/subject/experiment don't need to be assigned when archiving since they were already assigned during prearchiving:

for f in range(len(prearchive_session_to_archive)):
    print(f'Moving file {f+1} of {len(prearchive_session_to_archive)} to the archive...')
    archive_session = prearchive_session_to_archive[f].archive(project=projectLabel, \
                                                               subject=subjectLabel, \
                                                               experiment=experimentLabel)

In [80]:
# Only archive the desired files:

# Presumably I don't have to assign the subject and experiment again...
#archive_session = prearchive_session_to_archive.archive(subject=subjectLabel, experiment=experimentLabel)

for f in range(len(prearchive_session_to_archive)):
    print(f'Moving file {f+1} of {len(prearchive_session_to_archive)} to the archive...')
    
    #archive_session = prearchive_session_to_archive[f].archive()
    archive_session = prearchive_session_to_archive[f].archive(overwrite='append')

[2020-02-17 11:31:29,910] DEBUG: session:391 >> POST URI http://10.1.1.17/data/services/archive?src=%2Fdata%2Fprearchive%2Fprojects%2FBrainTumorProg_copy%2F20200217_112830490%2FPGM_002_MR_1&overwrite=append
[2020-02-17 11:31:29,911] DEBUG: session:393 >> POST DATA None


Moving file 1 of 9 to the archive...


[2020-02-17 11:31:30,259] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 11:31:30,260] DEBUG: session:769 >> Fetching object /data/archive/projects/BrainTumorProg_copy/subjects/PGM-002_Feb17/experiments/PGM-002_Feb17_MR_1 from cache
[2020-02-17 11:31:30,261] DEBUG: session:391 >> POST URI http://10.1.1.17/data/services/archive?src=%2Fdata%2Fprearchive%2Fprojects%2FBrainTumorProg_copy%2F20200217_112833178%2FPGM_002_MR_1&overwrite=append
[2020-02-17 11:31:30,261] DEBUG: session:393 >> POST DATA None
[2020-02-17 11:31:30,376] DEBUG: session:307 >> Received response with status code: 409


Moving file 2 of 9 to the archive...


XNATResponseError: Invalid response from XNATSession for url http://10.1.1.17/data/services/archive?src=%2Fdata%2Fprearchive%2Fprojects%2FBrainTumorProg_copy%2F20200217_112833178%2FPGM_002_MR_1&overwrite=append (status 409):
<html>
<head>
   <title>Status page</title>
</head>
<body>
<h3>Session processing may already be in progress: AutoRun.  Concurrent modification is discouraged.</h3><p>You can get technical details <a href="http://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html#sec10.4.10">here</a>.<br>
Please continue your visit at our <a href="/">home page</a>.
</p>
</body>
</html>


# Current Warnings
## CONFLICT-14: Session already exists with matching files.
## CONFLICT-16: Session already contains a scan (11) with the same UID and number.

In [89]:
# Remove the "archive_session" variable - maybe that's the source of the issue:

for f in range(len(prearchive_session_to_archive)):
    print(f'Moving file {f+1} of {len(prearchive_session_to_archive)} to the archive...')
    
    #prearchive_session_to_archive[f].archive()
    prearchive_session_to_archive[f].archive(overwrite='append')

[2020-02-17 12:51:16,064] DEBUG: session:391 >> POST URI http://10.1.1.17/data/services/archive?src=%2Fdata%2Fprearchive%2Fprojects%2FBrainTumorProg_copy%2F20200217_112831518%2FPGM_002_MR_1&overwrite=append
[2020-02-17 12:51:16,065] DEBUG: session:393 >> POST DATA None
[2020-02-17 12:51:16,174] DEBUG: session:307 >> Received response with status code: 409


Moving file 1 of 8 to the archive...


XNATResponseError: Invalid response from XNATSession for url http://10.1.1.17/data/services/archive?src=%2Fdata%2Fprearchive%2Fprojects%2FBrainTumorProg_copy%2F20200217_112831518%2FPGM_002_MR_1&overwrite=append (status 409):
<html>
<head>
   <title>Status page</title>
</head>
<body>
<h3>Session already exists with matching files.</h3><p>You can get technical details <a href="http://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html#sec10.4.10">here</a>.<br>
Please continue your visit at our <a href="/">home page</a>.
</p>
</body>
</html>


No change.

In [90]:
prearchive_session_to_archive

[<PrearchiveSession BrainTumorProg_copy/20200217_112831518/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112833178/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112829964/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112832580/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112831005/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112832043/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112834623/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_112833943/PGM-002_Feb17_MR_1>]

### I got an error when I ran the command above because one of the prearchived files was moved to the archive the first entry no longer existed. So prearchive_session_to_archive had to be redefined:

In [84]:
prearchive_session_to_archive = session.prearchive.find(project=projectLabel, \
                                                        subject=subjectLabel, \
                                                        session=experimentLabel, \
                                                        status='READY')

[2020-02-17 11:51:01,142] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy?format=json
[2020-02-17 11:51:01,156] DEBUG: session:307 >> Received response with status code: 200


In [85]:
len(prearchive_session_to_archive)

8

### Need to avoid trying to archive items that have already been archived:

In [91]:


status = 'True' # initial value

while status:
    # Update prearchive_session_to_archive:
    prearchive_session_to_archive = session.prearchive.find(project=projectLabel, \
                                                            subject=subjectLabel, \
                                                            session=experimentLabel, \
                                                            status='READY')
    
    #print('prearchive_session_to_archive = ', prearchive_session_to_archive)
    
    print('Number of files in prearchive_session_to_archive =', len(prearchive_session_to_archive), '\n')
    
    if len(prearchive_session_to_archive) == 0:
        status = 'False'
        break
        
    # Move each file in prearchive_session_to_archive to the archive:    
    for f in range(len(prearchive_session_to_archive)):
        print(f'Moving file {f+1} of {len(prearchive_session_to_archive)} to the archive...')
        
        prearchive_session_to_archive[f].archive(project=projectLabel, \
                                                 subject=subjectLabel, \
                                                 experiment=experimentLabel, \
                                                 overwrite='append')



[2020-02-17 12:54:45,128] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy?format=json
[2020-02-17 12:54:45,138] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 12:54:45,141] DEBUG: session:391 >> POST URI http://10.1.1.17/data/services/archive?src=%2Fdata%2Fprearchive%2Fprojects%2FBrainTumorProg_copy%2F20200217_112831518%2FPGM_002_MR_1&overwrite=append&project=BrainTumorProg_copy&subject=PGM-002_Feb17&session=PGM-002_Feb17_MR_1
[2020-02-17 12:54:45,142] DEBUG: session:393 >> POST DATA None
[2020-02-17 12:54:45,231] DEBUG: session:307 >> Received response with status code: 409


Number of files in prearchive_session_to_archive = 8
Moving file 1 of 8 to the archive...


XNATResponseError: Invalid response from XNATSession for url http://10.1.1.17/data/services/archive?src=%2Fdata%2Fprearchive%2Fprojects%2FBrainTumorProg_copy%2F20200217_112831518%2FPGM_002_MR_1&overwrite=append&project=BrainTumorProg_copy&subject=PGM-002_Feb17&session=PGM-002_Feb17_MR_1 (status 409):
<html>
<head>
   <title>Status page</title>
</head>
<body>
<h3>Session already exists with matching files.</h3><p>You can get technical details <a href="http://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html#sec10.4.10">here</a>.<br>
Please continue your visit at our <a href="/">home page</a>.
</p>
</body>
</html>


### That didn't help.  

### I found this thread:

#### https://groups.google.com/forum/#!topic/xnat_discussion/XG15WLydPkU

### so will look at Study Instance UIDs:

In [88]:
# Loop for each file name in dicomFnames:
for f in range(len(dicomFnames)):   
    # Read in the dicom file:
    dicom = pydicom.dcmread(dicomFpaths[f])
    
    print('Study Instance UID = ', dicom.StudyInstanceUID)
    

Study Instance UID =  1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313
Study Instance UID =  1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313
Study Instance UID =  1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313
Study Instance UID =  1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313
Study Instance UID =  1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313
Study Instance UID =  1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313
Study Instance UID =  1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313
Study Instance UID =  1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313
Study Instance UID =  1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313
Study Instance UID =  1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313
Study Instance UID =  1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313
Study Instance UID =  1.3.6.1.4.1.14519.5.2

### As with Ray's comments, my Study Instance UIDs are all the same.  I posted another question to this 2018 post:

#### https://groups.google.com/forum/#!topic/xnat_discussion/JfR_CYasSrM

### I selected the two files that had conflicts and rebuilt them.  See if that makes any difference:

In [92]:

for f in range(len(prearchive_session_to_archive)):
    print(f'Moving file {f+1} of {len(prearchive_session_to_archive)} to the archive...')
    
    #prearchive_session_to_archive[f].archive()
    prearchive_session_to_archive[f].archive(overwrite='append')

[2020-02-17 13:11:24,225] DEBUG: session:391 >> POST URI http://10.1.1.17/data/services/archive?src=%2Fdata%2Fprearchive%2Fprojects%2FBrainTumorProg_copy%2F20200217_112831518%2FPGM_002_MR_1&overwrite=append
[2020-02-17 13:11:24,225] DEBUG: session:393 >> POST DATA None


Moving file 1 of 8 to the archive...


[2020-02-17 13:11:24,491] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 13:11:24,491] DEBUG: session:738 >> Type unknown, fetching data to get type
[2020-02-17 13:11:24,493] DEBUG: session:334 >> GET URI http://10.1.1.17/data/archive/projects/BrainTumorProg_copy/subjects/PGM-002_Feb17/experiments/PGM_002_MR_1?format=json
[2020-02-17 13:11:24,571] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 13:11:24,572] DEBUG: session:746 >> Looking up type xnat:mrSessionData [str]
[2020-02-17 13:11:24,572] DEBUG: session:753 >> Creating object of type <class 'xnat_gen_4315f9d139004d953d804b4ecc4dd62a.MrSessionData'>
[2020-02-17 13:11:24,573] DEBUG: session:391 >> POST URI http://10.1.1.17/data/services/archive?src=%2Fdata%2Fprearchive%2Fprojects%2FBrainTumorProg_copy%2F20200217_112833178%2FPGM_002_MR_1&overwrite=append
[2020-02-17 13:11:24,574] DEBUG: session:393 >> POST DATA None
[2020-02-17 13:11:24,656] DEBUG: session:307 >> Received respons

Moving file 2 of 8 to the archive...


XNATResponseError: Invalid response from XNATSession for url http://10.1.1.17/data/services/archive?src=%2Fdata%2Fprearchive%2Fprojects%2FBrainTumorProg_copy%2F20200217_112833178%2FPGM_002_MR_1&overwrite=append (status 409):
<html>
<head>
   <title>Status page</title>
</head>
<body>
<h3>Session processing may already be in progress: AutoRun.  Concurrent modification is discouraged.</h3><p>You can get technical details <a href="http://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html#sec10.4.10">here</a>.<br>
Please continue your visit at our <a href="/">home page</a>.
</p>
</body>
</html>


### It moved the first of 8 but not where I was expecting it.  It moved it to a new MR Session with the label "PGM_002_MR_1" instead of "PGM-002_Feb17_MR_1".  So maybe I need to assign the REST variables when moving to the archive even though I had done so when prearchiving them...

### I've rebuilt the file in conflict, then need to redefine "prearchive_session_to_archive":

In [96]:
prearchive_session_to_archive = session.prearchive.find(project=projectLabel, \
                                                        subject=subjectLabel, \
                                                        session=experimentLabel, \
                                                        status='READY')

[2020-02-17 13:20:37,692] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy?format=json
[2020-02-17 13:20:37,701] DEBUG: session:307 >> Received response with status code: 200


In [97]:
# Move the selected prearchived files to archive:

for f in range(len(prearchive_session_to_archive)):
    print(f'Moving file {f+1} of {len(prearchive_session_to_archive)} to the archive...')
    
    #prearchive_session_to_archive[f].archive()
    prearchive_session_to_archive[f].archive(project=projectLabel, \
                                             subject=subjectLabel, \
                                             experiment=experimentLabel, \
                                             overwrite='append')

[2020-02-17 13:20:40,470] DEBUG: session:391 >> POST URI http://10.1.1.17/data/services/archive?src=%2Fdata%2Fprearchive%2Fprojects%2FBrainTumorProg_copy%2F20200217_112833178%2FPGM_002_MR_1&overwrite=append&project=BrainTumorProg_copy&subject=PGM-002_Feb17&session=PGM-002_Feb17_MR_1
[2020-02-17 13:20:40,471] DEBUG: session:393 >> POST DATA None
[2020-02-17 13:20:40,547] DEBUG: session:307 >> Received response with status code: 409


Moving file 1 of 7 to the archive...


XNATResponseError: Invalid response from XNATSession for url http://10.1.1.17/data/services/archive?src=%2Fdata%2Fprearchive%2Fprojects%2FBrainTumorProg_copy%2F20200217_112833178%2FPGM_002_MR_1&overwrite=append&project=BrainTumorProg_copy&subject=PGM-002_Feb17&session=PGM-002_Feb17_MR_1 (status 409):
<html>
<head>
   <title>Status page</title>
</head>
<body>
<h3>Session already exists with matching files.</h3><p>You can get technical details <a href="http://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html#sec10.4.10">here</a>.<br>
Please continue your visit at our <a href="/">home page</a>.
</p>
</body>
</html>


### So rebuilding hasn't helped.

# I've not been able to upload more than one scan by importing the files individually.  Now try to upload an entire session in a single .zip file.

### I zipped all files into a single file "files.zip" and deleted all files within the prearchive that belong to subject PGM-002_Feb17.

In [102]:
zippedDicomFpath = os.path.join(dicomDLdir, 'files.zip')
zippedDicomFpath

'C:\\Code\\WP1.3_multiple_modalities\\trying stuff\\XNAT downloads\\BrainTumorProg\\PGM-002\\0\\0\\PGM-002_MR_1\\scans\\11-T1post\\resources\\DICOM\\files\\files.zip'

In [104]:
os.path.isfile(zippedDicomFpath)

True

In [99]:
prearchive_session = session.services.import_(zippedDicomFpath, \
                                              project=projectLabel, \
                                              subject=subjectLabel, \
                                              experiment=experimentLabel, \
                                              destination='/prearchive')

[2020-02-17 13:52:10,017] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy&subject=PGM-002_Feb17&session=PGM-002_Feb17_MR_1
[2020-02-17 13:52:10,073] DEBUG: session:307 >> Received response with status code: 500


XNATUploadError: Upload failed after 1 attempts! Status code 500, response text 

### In the example provided for the XNATpy tutorial, the importing is done with assigning only the project and destination. When moving from prearchive to archive the subject and experiment are assigned.  Try that:

In [103]:
prearchive_session = session.services.import_(zippedDicomFpath, \
                                              project=projectLabel, \
                                              destination='/prearchive')

[2020-02-17 14:04:36,838] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy
[2020-02-17 14:04:36,881] DEBUG: session:307 >> Received response with status code: 500


XNATUploadError: Upload failed after 1 attempts! Status code 500, response text <html>
<head>
   <title>Status page</title>
</head>
<body>
<h3>Index: 0, Size: 0</h3><p>You can get technical details <a href="http://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html#sec10.5.1">here</a>.<br>
Please continue your visit at our <a href="/">home page</a>.
</p>
</body>
</html>


### Do as above but without sending it to the prearchive:

In [105]:
prearchive_session = session.services.import_(zippedDicomFpath, \
                                              project=projectLabel)

[2020-02-17 14:45:16,693] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?project=BrainTumorProg_copy
[2020-02-17 14:45:16,744] DEBUG: session:307 >> Received response with status code: 500


XNATUploadError: Upload failed after 1 attempts! Status code 500, response text 

### Likewise, try uploading individual DICOMs but without the prearchive step:

In [107]:
# Upload directly to the archive the DICOMs that have contour data:
for f in range(len(inds)):
    print(f'Uploading file {f+1} of {len(inds)} to the archive...')
    
    session.services.import_(dicomFpaths[inds[f]], \
                             project=projectLabel, \
                             subject=subjectLabel, \
                             experiment=experimentLabel)

[2020-02-17 14:48:19,444] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?project=BrainTumorProg_copy&subject=PGM-002_Feb17&session=PGM-002_Feb17_MR_1


Uploading file 1 of 9 to the archive...


[2020-02-17 14:48:20,432] DEBUG: session:307 >> Received response with status code: 200
[2020-02-17 14:48:20,433] DEBUG: session:769 >> Fetching object /data/archive/projects/BrainTumorProg_copy/subjects/PGM-002_Feb17/experiments/PGM-002_Feb17_MR_1 from cache
[2020-02-17 14:48:20,434] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?project=BrainTumorProg_copy&subject=PGM-002_Feb17&session=PGM-002_Feb17_MR_1


Uploading file 2 of 9 to the archive...


[2020-02-17 14:48:21,155] DEBUG: session:307 >> Received response with status code: 409


XNATUploadError: Upload failed after 1 attempts! Status code 409, response text <html>
<head>
   <title>Status page</title>
</head>
<body>
<h3>Session processing may already be in progress: AutoRun.  Concurrent modification is discouraged.</h3><p>You can get technical details <a href="http://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html#sec10.4.10">here</a>.<br>
Please continue your visit at our <a href="/">home page</a>.
</p>
</body>
</html>


### Trying to upload directly to the archive made no difference.